In [8]:
import os
import re
import shutil
import numpy as np
import pandas as pd 
from tqdm import tqdm

In [2]:
root_name = '/data/h-oiso/in_the_wild/'
print(len(os.listdir(root_name)))

31781


In [11]:
original_meta = pd.read_csv(os.path.join(root_name, 'meta.csv'))
original_meta.replace('bona-fide', 'bonafide', inplace=True)
original_meta.rename(columns={'file': 'utt', 'speaker': 'target'}, inplace=True)
original_meta

,utt,target,label
0,0.wav,Alec Guinness,spoof
1,1.wav,Alec Guinness,spoof
2,2.wav,Barack Obama,spoof
3,3.wav,Alec Guinness,spoof
4,4.wav,Christopher Hitchens,bonafide
...,...,...,...
31774,31774.wav,Boris Johnson,bonafide
31775,31775.wav,Donald Trump,bonafide
31776,31776.wav,Donald Trump,bonafide
31777,31777.wav,Alan Watts,bonafide


In [12]:
columns = ['-', 'utt', 'language', 'source', 'method', 'target', 'label', 'phase']

In [13]:
def delete_ext(series):
    for i, filename in tqdm(enumerate(series)):
        series[i] = filename[:-4]
    return series

def delete_space(series):
    for i, name in tqdm(enumerate(series)):
        series[i] = name.replace(" ", "_")
    return series

In [14]:
original_meta['-'] = ['-']*len(original_meta) # first col
delete_ext(original_meta['utt']) # utt
original_meta['language'] = 'ENG' # language
original_meta['source'] = ['-']*len(original_meta) # source
original_meta['method'] = ['-']*len(original_meta) # method
delete_space(original_meta['target']) # target
# label is ok
original_meta['phase'] = ['-']*len(original_meta) # phase
original_meta = original_meta.reindex(columns=columns)

31779it [00:16, 1954.97it/s]
31779it [00:15, 2025.72it/s]


In [15]:
original_meta

,-,utt,language,source,method,target,label,phase
0,-,0,ENG,-,-,Alec_Guinness,spoof,-
1,-,1,ENG,-,-,Alec_Guinness,spoof,-
2,-,2,ENG,-,-,Barack_Obama,spoof,-
3,-,3,ENG,-,-,Alec_Guinness,spoof,-
4,-,4,ENG,-,-,Christopher_Hitchens,bonafide,-
...,...,...,...,...,...,...,...,...
31774,-,31774,ENG,-,-,Boris_Johnson,bonafide,-
31775,-,31775,ENG,-,-,Donald_Trump,bonafide,-
31776,-,31776,ENG,-,-,Donald_Trump,bonafide,-
31777,-,31777,ENG,-,-,Alan_Watts,bonafide,-


In [16]:
original_meta['label'].value_counts()

bonafide    19963
spoof       11816
Name: label, dtype: int64

In [17]:
train, dev, test = np.split(original_meta.sample(frac=1), [int(.4*len(original_meta)), int(.6*len(original_meta))])

In [18]:
train['label'].value_counts()

bonafide    7935
spoof       4776
Name: label, dtype: int64

In [19]:
dev['label'].value_counts()

bonafide    3970
spoof       2386
Name: label, dtype: int64

In [20]:
test['label'].value_counts()

bonafide    8058
spoof       4654
Name: label, dtype: int64

In [21]:
train['phase'] = ['train']*len(train)
dev['phase'] = ['dev']*len(dev)
test['phase'] = ['test']*len(test)

In [22]:
cm_path = '/home/h-oiso/AudioFake/SSL_Anti-spoofing/LA-keys-stage-1/keys/CM'
train.to_csv(os.path.join(cm_path, 'train_metadata_In_The_Wild.txt'), sep=' ', header=None, index=False)
dev.to_csv(os.path.join(cm_path, 'dev_metadata_In_The_Wild.txt'), sep=' ', header=None, index=False)
test.to_csv(os.path.join(cm_path, 'test_metadata_In_The_Wild.txt'), sep=' ', header=None, index=False)